# T81 Notebook 02 — Toy T81VM Trace Walkthrough

This notebook provides a **toy VM** and step-by-step traces to build intuition for:

- Instruction-level execution.
- Register and program counter (PC) state.
- Simple branching and loops.

This is **not** the canonical T81VM or TISC implementation.  
It is a small, Python-based model inspired by the idea of a ternary-native VM that you can inspect and modify easily.


## 0. Context and Limitations

For the real thing, see:

- `spec/t81vm-spec.md`
- `spec/tisc-spec.md`
- Legacy VM code under `legacy/hanoivm/src/`
- Emerging IR/VM headers under `include/t81/ir/`

This notebook:

- Uses a **minimal instruction set**: `NOP`, `LOAD`, `ADD`, `SUB`, `JMP`, `JZ`, `HALT`, `PRINT`.
- Stores values as Python integers, but exposes optional **balanced ternary views** for inspection.
- Focuses on clarity and traceability rather than performance or feature completeness.


## 1. Balanced Ternary View Helpers

We reuse a small subset of the balanced ternary helpers so that VM state can be shown in both integer and ternary form.


In [ ]:
from __future__ import annotations

from dataclasses import dataclass, field
from typing import List, Dict, Callable, Optional, Sequence

BT_SYMBOLS = {
    -1: "-",
     0: "0",
     1: "+",
}
BT_SYMBOL_TO_VALUE = {v: k for k, v in BT_SYMBOLS.items()}


def int_to_bt_digits(n: int) -> List[int]:
    if n == 0:
        return [0]

    digits: List[int] = []
    x = n
    while x != 0:
        x, remainder = divmod(x, 3)
        if remainder == 2:
            remainder = -1
            x += 1
        digits.append(remainder)

    digits.reverse()
    while len(digits) > 1 and digits[0] == 0:
        digits.pop(0)
    return digits


def bt_digits_to_str(digits: Sequence[int]) -> str:
    return "".join(BT_SYMBOLS[d] for d in digits)


def int_to_bt_str(n: int) -> str:
    return bt_digits_to_str(int_to_bt_digits(n))


## 2. Toy Instruction Set and VM Structure

We define a very small instruction set:

- `NOP` — no operation.
- `LOAD r, imm` — `R[r] = imm`.
- `ADD r_dst, r_a, r_b` — `R[r_dst] = R[r_a] + R[r_b]`.
- `SUB r_dst, r_a, r_b` — `R[r_dst] = R[r_a] - R[r_b]`.
- `JMP target` — unconditional jump.
- `JZ r, target` — jump if `R[r] == 0`.
- `HALT` — stop execution.
- `PRINT r` — append `R[r]` to an output log.

Registers are named `"R0"`, `"R1"`, `"R2"`, `"R3"` for simplicity.


In [ ]:
Opcode = str

@dataclass
class Instruction:
    op: Opcode
    args: List[int]  # registers are encoded as small ints (0..3) or immediate/PC targets


@dataclass
class ToyVM:
    program: List[Instruction]
    registers: List[int] = field(default_factory=lambda: [0, 0, 0, 0])
    pc: int = 0
    halted: bool = False
    outputs: List[int] = field(default_factory=list)

    def step(self) -> None:
        if self.halted:
            return
        if not (0 <= self.pc < len(self.program)):
            raise RuntimeError(f"PC out of range: {self.pc}")

        instr = self.program[self.pc]
        op, args = instr.op, instr.args

        if op == "NOP":
            self.pc += 1

        elif op == "LOAD":
            r, imm = args
            self.registers[r] = imm
            self.pc += 1

        elif op == "ADD":
            rd, ra, rb = args
            self.registers[rd] = self.registers[ra] + self.registers[rb]
            self.pc += 1

        elif op == "SUB":
            rd, ra, rb = args
            self.registers[rd] = self.registers[ra] - self.registers[rb]
            self.pc += 1

        elif op == "JMP":
            target, = args
            self.pc = target

        elif op == "JZ":
            r, target = args
            if self.registers[r] == 0:
                self.pc = target
            else:
                self.pc += 1

        elif op == "PRINT":
            r, = args
            self.outputs.append(self.registers[r])
            self.pc += 1

        elif op == "HALT":
            self.halted = True

        else:
            raise RuntimeError(f"Unknown opcode: {op}")

    def run(self, max_steps: int = 1000) -> None:
        steps = 0
        while not self.halted and steps < max_steps:
            self.step()
            steps += 1
        if steps >= max_steps and not self.halted:
            raise RuntimeError("Max steps exceeded; possible infinite loop.")

    def dump_state(self) -> None:
        print(f"PC = {self.pc}")
        for i, val in enumerate(self.registers):
            print(f"  R{i}: {val:4d}  (bt={int_to_bt_str(val)})")
        print(f"  outputs: {self.outputs}")


## 3. Example Program 1 — Sum 1..N

We build a small program to compute the sum `1 + 2 + ... + N`.

Strategy:

- `R0` — loop counter `i`.
- `R1` — accumulator `sum`.
- `R2` — constant `1`.
- `R3` — target `N`.

Pseudocode:

```text
R0 = 0
R1 = 0
R2 = 1
R3 = N

loop:
  R0 = R0 + R2        # i += 1
  R1 = R1 + R0        # sum += i
  Rtemp = R3 - R0
  if Rtemp == 0: goto done
  goto loop

done:
  PRINT R1
  HALT
```

We encode this using our small instruction set and then trace it step-by-step.


In [ ]:
def make_sum_1_to_n_program(N: int) -> ToyVM:
    prog: List[Instruction] = []

    # 0: LOAD R0, 0
    prog.append(Instruction("LOAD", [0, 0]))
    # 1: LOAD R1, 0
    prog.append(Instruction("LOAD", [1, 0]))
    # 2: LOAD R2, 1
    prog.append(Instruction("LOAD", [2, 1]))
    # 3: LOAD R3, N
    prog.append(Instruction("LOAD", [3, N]))

    # loop_start = 4
    loop_start = len(prog)

    # R0 = R0 + R2
    prog.append(Instruction("ADD", [0, 0, 2]))  # 4
    # R1 = R1 + R0
    prog.append(Instruction("ADD", [1, 1, 0]))  # 5
    # Rtemp = R3 - R0  (reuse R2 as temp)
    prog.append(Instruction("SUB", [2, 3, 0]))  # 6
    # if R2 == 0: goto done
    # done is at index len(prog) + 3 (PRINT, HALT)
    # placeholder; we will patch once we know done index
    jz_index = len(prog)
    prog.append(Instruction("JZ", [2, 0]))      # 7
    # goto loop_start
    prog.append(Instruction("JMP", [loop_start]))  # 8

    done_index = len(prog)
    prog[jz_index].args[1] = done_index  # patch JZ target

    prog.append(Instruction("PRINT", [1]))  # done_index
    prog.append(Instruction("HALT", []))    # done_index + 1

    return ToyVM(program=prog)


vm = make_sum_1_to_n_program(5)

step = 0
while not vm.halted and step < 50:
    print(f"\n--- step {step} ---")
    vm.dump_state()
    vm.step()
    step += 1

print("\nFinal state:")
vm.dump_state()


You can change `N` in `make_sum_1_to_n_program(N)` to see how the trace evolves and how many steps are required for different loop bounds.


## 4. Example Program 2 — Simple Branching

This example shows a bare-bones branching pattern:

- If a register is zero, jump to one label.
- Otherwise, fall through to another path.

We use the toy VM to illustrate branching and PC changes clearly.


In [ ]:
def make_branch_demo_program() -> ToyVM:
    prog: List[Instruction] = []

    # 0: LOAD R0, 0   ; change to 1 to take the other branch
    prog.append(Instruction("LOAD", [0, 0]))

    # 1: JZ R0, 4     ; if R0 == 0, jump to index 4
    prog.append(Instruction("JZ", [0, 4]))

    # 2: LOAD R1, 111 ; else-branch
    prog.append(Instruction("LOAD", [1, 111]))
    # 3: JMP 5        ; skip then-branch
    prog.append(Instruction("JMP", [5]))

    # 4: LOAD R1, 222 ; then-branch (R0 == 0)
    prog.append(Instruction("LOAD", [1, 222]))

    # 5: PRINT R1
    prog.append(Instruction("PRINT", [1]))
    # 6: HALT
    prog.append(Instruction("HALT", []))

    return ToyVM(program=prog)


vm2 = make_branch_demo_program()

step = 0
while not vm2.halted and step < 20:
    print(f"\n--- step {step} ---")
    vm2.dump_state()
    vm2.step()
    step += 1

print("\nFinal state:")
vm2.dump_state()


Try modifying the immediate value in `LOAD R0, imm` (instruction 0) from `0` to `1` and re-running the cell to see how control flow and outputs change.


## 5. Extending This Toy VM Toward T81

Ways you can extend this notebook to better mirror the T81 ecosystem:

1. **Ternary-native registers**
   - Store values as balanced ternary digit lists instead of Python ints.
   - Implement ALU operations in terms of the balanced ternary helpers from Notebook 01.

2. **Richer instruction set**
   - Add instructions for memory access, comparison, and ternary-specific flags.
   - Mirror some of the TISC opcodes defined in `spec/tisc-spec.md` (or your current draft).

3. **Trace visualizations**
   - Log state changes per step into a list, then render them as tables or graphs.
   - Highlight when certain invariants are violated or key events occur (e.g., loop boundaries).

4. **Bridging to real T81VM**
   - Once you have C++ bindings (e.g., via `pybind11`), you can replace this toy VM with calls into the actual VM, keeping the trace and visualization code.

This notebook is intentionally compact and opinionated so that it can live alongside the formal specs as an **executable, inspectable mental model** of instruction-level behavior.
